In [3]:
#Setting up the Notebook
import pandas as pd
import numpy as np
from numpy.random import randn

In [4]:
#Setting up the DataFrames
trackingDF = pd.read_csv("tracking_gameId_2017090700.csv") #This is for one game, we'll need the other tracking data from the repo
playersDF = pd.read_csv("players.csv")
gamesDF = pd.read_csv("games.csv")
playsDF = pd.read_csv("plays.csv")

trackingDF.dropna(axis=0, inplace=True) #Directly remove any row that contains null information
trackingDF['nflId'] = trackingDF['nflId'].astype(int) #Need to unfloat the floats to match nflId
playersDF.dropna(axis=0, inplace=True)
gamesDF.dropna(axis=0, inplace=True)
playsDF.dropna(axis=0,inplace=True)

In [5]:
#See the columns (Series)
print("\nTracking Columns\n")
print(trackingDF.columns)
print("\nPlayers Columns\n")
print(playersDF.columns)
print("\nGames Columns\n")
print(gamesDF.columns)
print("\nPlays Columns\n")
print(playsDF.columns)


Tracking Columns

Index(['time', 'x', 'y', 's', 'dis', 'dir', 'event', 'nflId', 'displayName',
       'jerseyNumber', 'team', 'frame.id', 'gameId', 'playId'],
      dtype='object')

Players Columns

Index(['nflId', 'FirstName', 'LastName', 'PositionAbbr', 'EntryYear',
       'DraftRound', 'DraftNumber', 'Height', 'Weight', 'College'],
      dtype='object')

Games Columns

Index(['season', 'week', 'gameDate', 'gameId', 'gameTimeEastern', 'HomeScore',
       'VisitorScore', 'homeTeamAbbr', 'visitorTeamAbbr', 'homeDisplayName',
       'visitorDisplayName', 'Stadium', 'Location', 'StadiumType', 'Turf',
       'GameLength', 'GameWeather', 'Temperature', 'Humidity', 'WindSpeed',
       'WindDirection'],
      dtype='object')

Plays Columns

Index(['gameId', 'playId', 'quarter', 'GameClock', 'down', 'yardsToGo',
       'possessionTeam', 'yardlineSide', 'yardlineNumber', 'offenseFormation',
       'personnel.offense', 'defendersInTheBox', 'numberOfPassRushers',
       'personnel.defense', 'Ho

In [43]:
#Need PlayersDF to have a team associated pull data from Tracking to match
indices = ['nflId', 'gameId', 'team'] #nflId is unique which is tied to player, gameId and team will help id who was playing
playersTeamDF = trackingDF[indices]
playersTeamDF.head()

nflId      gameId  team
4    2495340  2017090700  away
44   2495340  2017090700  away
81   2495340  2017090700  away
87   2495340  2017090700  away
102  2507948  2017090700  away

In [44]:
#Since an nflId is either away or home, we know that the gameId associated with it has the team name now we need the team name
indices = ['gameId', 'homeTeamAbbr' , 'visitorTeamAbbr']
gameTeamDF = gamesDF[indices]
gameTeamDF.head()
#Note, this is why we need the tracking data from other games. With the current tracking data we can only tie two teams
#See the gameId, it's the only id that matches the players
onlyDF = gameTeamDF.iloc[0] #gameId is the one we need

homeTeam = onlyDF['homeTeamAbbr']
awayTeam = onlyDF['visitorTeamAbbr']
gameId = onlyDF['gameId']

In [48]:
#This ties team names to players
playersTeamDF['teamName'] = np.where(playersTeamDF['team']=='away', awayTeam, homeTeam)
uniquePlayers = playersTeamDF.drop_duplicates(subset='nflId', keep='first')
uniquePlayers.head()

C:\Users\Kendrick-Laptop\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


nflId      gameId  team teamName
4    2495340  2017090700  away       KC
102  2507948  2017090700  away       KC
200  2541187  2017090700  away       KC
298  2543563  2017090700  away       KC
396  2543638  2017090700  away       KC

In [49]:
#Have to merge this data based on the nflId2
uniquePlayers = uniquePlayers.drop(columns=['team', 'gameId'])
addedTeamsDF = pd.merge(playersDF, uniquePlayers, how='left', on='nflId')
addedTeamsDF.dropna(axis=0,inplace=True)
addedTeamsDF

nflId    FirstName         LastName PositionAbbr  EntryYear  \
15      71251      Patrick            Chung           SS       2009   
26      79611          Roy           Miller           NT       2009   
41    2506340    Alexander            Smith           QB       2005   
44    2506353      Derrick          Johnson          ILB       2005   
58    2495561         Alan           Branch           DT       2007   
61    2495621        David           Harris          OLB       2007   
101   2533062      Stephon          Gilmore           CB       2012   
109   2549896      Marquis          Flowers          OLB       2014   
130   2552488       Marcus           Peters           CB       2015   
140   2552424      Phillip          Dorsett           WR       2015   
143   2552674       Rakeem     Nunez-Roches           DT       2015   
159   2543637     Terrance         Mitchell           CB       2014   
162   2552483       Malcom            Brown           DT       2015   
173   2552255       Nelson             Rowe           CB       2015   
174   2552668     Mitchell            Morse            C       2015   
187   2552392       Jordan         Richards           SS       2015   
195   2552652    Christian           Conley           WR       2015   
214   2552278         Trey          Flowers           DE       2015   
223   2552320        Ramik           Wilson          ILB       2015   
232   2552563    Shaquille            Mason            G       2015   
251   2552701          Joe          Cardona           LS       2015   
271   2504211          Tom            Brady           QB       2000   
277   2506438       Dustin         Colquitt            P       2005   
292   2506922      Stephen       Gostkowski            K       2006   
327    496723        James            Berry           SS       2010   
342    494287        Devin         McCourty           FS       2010   
349    497240       Robert       Gronkowski           TE       2010   
399   2495232    Nathaniel           Solder            T       2011   
427   2495493       Justin          Houston          OLB       2011   
432   2495116        Allen           Bailey           DE       2011   
...       ...          ...              ...          ...        ...   
507   2533046       Dwayne            Allen           TE       2012   
566   2539213         Eric           Fisher            T       2013   
615   2540258       Travis            Kelce           TE       2013   
617   2540163       Bennie            Logan           DT       2013   
633   2541243        Duron           Harmon           FS       2013   
658   2539663         Mike        Gillislee           RB       2013   
667   2539265      Michael         Burkhead           RB       2013   
701   2543498      Brandin            Cooks           WR       2014   
703   2543494       Donald             Ford          OLB       2014   
717   2543699         Kyle          Van Noy          OLB       2014   
748   2543851      Phillip           Gaines           CB       2014   
763   2543868      Cassius            Marsh           DE       2014   
774   2543638   De'Anthony           Thomas           WR       2014   
779   2543773        James            White           RB       2014   
780   2543563        Kevin     Pierre-Louis          ILB       2014   
805   2549981      Kenneth            Acker           CB       2014   
811   2543862      Zachery           Fulton            G       2014   
812   2543816      Laurent  Duvernay-Tardif            G       2014   
856   2556369  Christopher            Jones           DE       2016   
891   2555281       Joseph           Thuney            G       2016   
913   2555173         Eric           Murray           FS       2016   
930   2555478     Demarcus         Robinson           WR       2016   
956   2556214       Tyreek             Hill           WR       2016   
985   2556119      Elandon          Roberts          MLB       2016   
988   2555145          Ted      

In [ ]:
#At this point, we have two teams but we need more tracking data and then rewrite the previous code to fix that.